In [13]:
type Kind = 
| Start
| End
| Step
type Position = {
    line: int
    col: int
    height: int
    kind: Kind
}
module Position =
    let value line col (character:char) =
        let toInt c = (c |> int) - 96
        let aPos = { line = line; col = col; height = -1; kind = Step }
        match character with
        | 'S' -> { aPos with height = 'a' |> toInt; kind = Start }
        | 'E' -> { aPos with height = 'z' |> toInt; kind = End }
        | _ -> { aPos with height = character |> toInt }

module Path =
    let mapIt values =
        values
        |> Seq.mapi (fun idx line -> 
            line 
            |> Seq.mapi (fun idy value -> value |> Position.value idx idy )
            |> List.ofSeq)
        |> List.ofSeq

    let start map =
        map
        |> Seq.concat
        |> Seq.filter (fun it -> match it.kind with | Kind.Start _ -> true | _ -> false)
        |> Seq.tryHead
    
    type Direction = UP | DOWN | LEFT | RIGHT
    
    let walk direction (map:Position list list) (position:Position) =
        let height = (map |> List.length) - 1
        let width = (map.[0] |> List.length) - 1

        let walkInto dest origin =
            if Math.Abs(dest.height - origin.height) <= 1 then
                Some dest
            else 
                None

        match direction with
        | DOWN when position.line < height ->
            position
            |> walkInto map.[position.line + 1].[position.col]
        | UP when position.line > 0 ->
            position
            |> walkInto map.[position.line - 1].[position.col]
        | LEFT when position.col < width ->
            position
            |> walkInto map.[position.line].[position.col + 1]
        | RIGHT when position.col > 0 ->
            position
            |> walkInto map.[position.line].[position.col - 1]
        | _ -> None
    

let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines( ResolutionFolder + "/testcase12.txt")

let map =
    lines
    |> Path.mapIt

let start = map |> Path.start

start |> Option.bind (Path.walk Path.Direction.DOWN map )

Value
Position line: 1 col: 0 height: 1 kind: Step
